# Predicting if a message is spam or not


In [1]:
#import required libraries 
import pandas as pd
import numpy as np

spam_data = pd.read_csv(r'C:\Users\sharm.LAPTOP-118C54MT\OneDrive - York University\Coursera\Course_4\Assignment_3\spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [2]:
#percentage of the spam documents in spam_data
(spam_data['target'].sum()/spam_data['target'].count())*100

13.406317300789663

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
#fit training data using a count vectorizer with default parameters and get the longest string
vect = CountVectorizer().fit(X_train)
longest_string = max(vect.get_feature_names(), key=len)
longest_string

'com1win150ppmx3age16subscription'

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
#fit a mulinomial naive bayes classifier model with smoothing alpha = 0.1. Find AUC score using the 
#transformed test data 
vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
clfrNB =  MultinomialNB(alpha=0.1)
clfrNB.fit(X_train_vectorized, y_train)
predictions = clfrNB.predict(vect.transform(X_test))
roc_auc_score(y_test,predictions)

0.9720812182741116

In [7]:
#fit and transform training data using Tfidf vectorizer with  default parameters.
#find 20 features with smallest and largest tf-idf 
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer().fit(X_train) #tfidf vectorizer
X_train_vectorized = vect.transform(X_train) #fitting on training data 
feature_names = np.array(vect.get_feature_names()) #get the featuress 
sorted_tfidf_index = (X_train_vectorized.max(0).toarray()[0]).argsort()
smallest_tfidf_feature = feature_names[sorted_tfidf_index[:20]] #smallest tfidf features 
largest_tfidf_feature = feature_names[sorted_tfidf_index[:-21:-1]]
smallest_tfidf_feature_dict = {'value' : smallest_tfidf_feature, 'index' : (X_train_vectorized.max(0).toarray()[0])
                                         [sorted_tfidf_index[:20]]}
largest_tfidf_feature_dict = {'value' : largest_tfidf_feature, 'index' : (X_train_vectorized.max(0).toarray()[0])
                                         [sorted_tfidf_index[:-21:-1]]}
smallest_tfidf_feature_df = pd.DataFrame(smallest_tfidf_feature_dict)
largest_tfidf_feature_df = pd.DataFrame(largest_tfidf_feature_dict)

smallest_tfidf_feature_df.sort_values(by = ['index', 'value'],inplace = True)
largest_tfidf_feature_df.sort_values(by = ['index', 'value'],inplace = True, ascending = [False, True])

print(pd.Series(smallest_tfidf_feature_df['value'].to_list(), index= smallest_tfidf_feature_df['index']),\
pd.Series(largest_tfidf_feature_df['value'].to_list(), index= largest_tfidf_feature_df['index']))

index
0.074475          aaniye
0.074475        athletic
0.074475            chef
0.074475       companion
0.074475      courageous
0.074475      dependable
0.074475      determined
0.074475    exterminator
0.074475          healer
0.074475        listener
0.074475       organizer
0.074475            pest
0.074475    psychiatrist
0.074475    psychologist
0.074475         pudunga
0.074475         stylist
0.074475     sympathetic
0.074475          venaam
0.091250          diwali
0.091250        mornings
dtype: object index
1.000000    146tf150p
1.000000          645
1.000000     anything
1.000000      anytime
1.000000      beerage
1.000000         done
1.000000           er
1.000000       havent
1.000000         home
1.000000          lei
1.000000         nite
1.000000           ok
1.000000         okie
1.000000        thank
1.000000        thanx
1.000000          too
1.000000        where
1.000000          yup
0.980166         tick
0.932702        blank
dtype: object


In [8]:
# AUC score after fitting data on tfidf vectorizer and then using multinomial Naive Vayes Classifier
vect = TfidfVectorizer(min_df=3).fit(X_train)
X_train_vectorized = vect.transform(X_train)
clfrNB =  MultinomialNB(alpha=0.1)
clfrNB.fit(X_train_vectorized, y_train)
predictions = clfrNB.predict(vect.transform(X_test))
roc_auc_score(y_test,predictions) 
    

0.9416243654822335

In [9]:
# average length of documents for not spam and spam documents 
spam = spam_data[spam_data['target']==1]
not_spam = spam_data[spam_data['target']!=1]
avg_spam = spam['text'].str.len().sum()/len(spam.index)
avg_notspam = not_spam['text'].str.len().sum()/len(not_spam.index) 
print(avg_notspam, avg_spam)

71.02362694300518 138.8661311914324


In [11]:
#this function combines new features into the training data.
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [12]:
from sklearn.svm import SVC
# fit and transform training data using tfidf vectorizer ignoring the document with a frequency lower than 5
#combine this document with a new feature the length of document, fit a support vector classification model
#with regularization c=1000.
#comput the AUC score 

vect = TfidfVectorizer(min_df=5).fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_train_final = add_feature(X_train_vectorized,X_train.str.len())
clfrSVM =  SVC(C=10000)
clfrSVM.fit(X_train_final, y_train)
predictions = clfrSVM.predict(add_feature(vect.transform(X_test),X_test.str.len()))
roc_auc_score(y_test,predictions) 
    

0.9661689557407943

In [14]:
#function to calculate length of the list for a vectorized string
def len_list(x):
    ind_len = []
    for i in x:
        ind_len.append(len(i))
    return sum(ind_len)

In [15]:
#average number of digits per document for not spam and spam documents
spam = spam_data[spam_data['target']==1]
not_spam = spam_data[spam_data['target']!=1]
spam_digit = spam['text'].str.findall(r'([0-9]+)').apply(len_list)
notspam_digit = not_spam['text'].str.findall(r'([0-9]+)').apply(len_list)
avg_spam = spam_digit.sum()/len(spam.index)
avg_notspam = notspam_digit.sum()/len(not_spam.index)
print(avg_notspam, avg_spam)

0.2992746113989637 15.759036144578314


In [16]:
from sklearn.linear_model import LogisticRegression
#fit and transform training data using Tfidf vectorizer ignoring documents with a frequency lower than 5
#add features the length of document and number of digits per document to the matrix 
#fit a logistic regression model with C=100 and calculate AUC.
vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_train_final = add_feature(X_train_vectorized,[X_train.str.len(),\
                                                X_train.str.findall(r'([0-9]+)').apply(len_list)])
clfrlreg =  LogisticRegression(C=100)
clfrlreg.fit(X_train_final, y_train)
predictions = clfrlreg.predict(add_feature(vect.transform(X_test),\
                                          [X_test.str.len(), X_test.str.findall(r'([0-9]+)').apply(len_list)]))

roc_auc_score(y_test,predictions) 
#much better accuracy than before 

C:\Users\sharm.LAPTOP-118C54MT\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9809793219360643

In [17]:
#average number of non-word characters per document for not spam and spam documents 
spam = spam_data[spam_data['target']==1]
not_spam = spam_data[spam_data['target']!=1]
spam_char = spam['text'].str.findall(r'(\W+)').apply(len_list)
notspam_char = not_spam['text'].str.findall(r'(\W+)').apply(len_list)
avg_spam = spam_char.sum()/len(spam.index)
avg_notspam = notspam_char.sum()/len(not_spam.index)
    
print(avg_notspam, avg_spam)

17.29181347150259 29.041499330655956


In [18]:
#fit and transform training data using a count vectorizer ignoring documents that have frequency lower than
#5 and using characer n-grams from n=2 to n=5 
#add features length of document, number of digits per document and number of non-word characters
#fit a logistic regression model with regularization C=100, compute AUC score 
#find 10 smallest and 10 largest coefficients from the model 
vect = TfidfVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_train_final = add_feature(X_train_vectorized,[X_train.str.len(),\
                                                X_train.str.findall(r'([0-9]+)').apply(len_list),\
                           X_train.str.findall(r'(\W+)').apply(len_list)])
clfrlreg =  LogisticRegression(C=100)
clfrlreg.fit(X_train_final, y_train)
predictions = clfrlreg.predict(add_feature(vect.transform(X_test),\
                                          [X_test.str.len(), X_test.str.findall(r'([0-9]+)').apply(len_list),\
                                          X_test.str.findall(r'(\W+)').apply(len_list)]))
AUC = roc_auc_score(y_test,predictions)
smallest = list(np.sort(clfrlreg.coef_[0])[:10])
largest = list(np.sort(clfrlreg.coef_[0])[:-11:-1])
print(AUC,smallest,largest)
#great the accuray seems really good after adding features and tweeking some parameters.

C:\Users\sharm.LAPTOP-118C54MT\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9733651087380949 [-3.47119919721989, -3.071198885849946, -3.0329634588744185, -2.794473948646245, -2.738412966772945, -2.6916916635565378, -2.6697308889533957, -2.5150605475477223, -2.45198471401025, -2.4501798093530587] [5.399780531301951, 4.724937924225051, 4.6293773893619425, 4.016249722603797, 3.937401048395025, 3.7942602042979883, 3.5290774094342408, 3.5118392561557057, 3.4282033484005368, 3.4005547897044752]
